In [1]:
import sys
sys.path.append('../../')
sys.path.append('../')

import fm
from torch.profiler import profile, record_function, ProfilerActivity

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
import torch.fx as fx
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [3]:
def gen_pattern_replace_and_matcher_for_FM(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

  class PatternClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = torch.nn.Embedding(1, 1)

    def forward(self,x):
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      sum_of_square = torch.sum(x ** 2, dim=1)
      ix = square_of_sum - sum_of_square    
      ix = torch.sum(ix, dim=1, keepdim=True)  
      return 0.5 * ix
  def _match(match,ori,pat):
    return True 
  # env  = utils.get_env(traced)
  # target_node = env[target_node_name]
  # target_node_mod = utils.get_target_mod(traced,target_node_name,"_")
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  

  
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      embedding_config = embed_node_module.weight.data.shape
      self.embed = nn.Embedding(embedding_config[0],embedding_config[1])
      self.embed.weight.data.copy_(embed_node_module.weight.data)
    def forward(self,x):
      redency_part = x[redency_part_slice] 
      unredency_part = x[unredency_part_slice] 
      redency_embed = self.embed(redency_part)
      unredency_embed = self.embed(unredency_part)
      redency_embed_sum = torch.sum(redency_embed,dim=0)
      unredency_embed_sum = torch.sum(unredency_embed,dim=1)
      square_of_sum = (redency_embed_sum + unredency_embed_sum) ** 2
      redency_embed_square_sum = torch.sum(redency_embed ** 2,dim=0)
      unredency_embed_square_sum = torch.sum(unredency_embed ** 2,dim=1)
      sum_of_square = redency_embed_square_sum + unredency_embed_square_sum
      ix = square_of_sum - sum_of_square
      ix = torch.sum(ix,dim = 1,keepdim=True)
      return 0.5 * ix
    
  
  return pattern,ReplacementClass(),_match

In [4]:
def workload_FM(num_field, prefix,dim = 64):
  # print(f"now gen workload of wdl with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}")
  model_ori = fm.FactorizationMachineModel([100 for i in range(num_field)],dim,)
  ori_traced = symbolic_trace(model_ori)
  
  model_modify = fm.FactorizationMachineModel([100 for i in range(num_field)],dim,)
  modify_traced = symbolic_trace(model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_FM(modify_traced,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      )
  matches = subgraph_rewriter.replace_pattern_with_filters(modify_traced, pattern, replace,[match])
  return ori_traced,modify_traced

In [5]:
def calculate_mean_and_variance_manual(data):
    n = len(data)
    mean = sum(data) / n
    variance = sum((x - mean) ** 2 for x in data) / n
    return mean, variance

In [7]:
def gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_FM):
  def run(model):
    t = torch.randint(low=0, high=88, size=(batch,num_field), dtype=torch.long).cuda()
    traced_model = torch.jit.trace(model.cuda(), t)
    compiled_model = torch.compile(traced_model, backend="inductor")
    compiled_model.eval()
    total_time = []
    for i in range(100):
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)  
        start_event.record()

        with torch.no_grad():
          soutput = compiled_model(t)
        end_event.record()
        torch.cuda.synchronize()
        # 计算并打印函数执行所需的时间
        elapsed_time = start_event.elapsed_time(end_event)
        total_time.append(elapsed_time)
    print(calculate_mean_and_variance_manual(total_time[2:]))
  print(f"now gen workload of fm with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}, batch :{batch}")
  ori, modify = workload_func(num_field,prefix,dim)
  run(ori)
  run(modify)

In [174]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :1024
(0.24496293965042854, 3.6901617091051304e-05)
(0.3211735505230573, 0.00016730831776382073)


In [175]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :2048
(0.3712995919037838, 7.78932647711557e-05)
(0.31460701841480876, 5.8910828594731324e-05)


In [176]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :4096
(0.6269322457362194, 6.004928869995184e-05)
(0.46117681690624784, 7.912373543456429e-05)


In [177]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 8192, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :8192
(1.140207683553501, 0.029961910003152414)
(0.7198233470624807, 6.236071498039282e-05)


In [178]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 110, prefix: 50, batch :1024
(0.36762840741751146, 4.3714208059248475e-05)
(0.31626677543533094, 8.753006318531525e-05)


In [179]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 110, prefix: 50, batch :2048
(0.6362145324142612, 0.05322876640533168)
(0.4471353453641035, 3.310831655977856e-05)


In [180]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 110, prefix: 50, batch :4096
(1.113210772981449, 6.382334816363714e-05)
(0.7152097918549363, 5.47502814092311e-05)


In [181]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 8192, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 110, prefix: 50, batch :8192
(2.075804111908893, 0.00012493646271312332)
(1.250894044126783, 0.00010837843005160968)


In [182]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :1024
(0.3116058792386736, 4.713469793604199e-05)
(0.3200287338422269, 0.00018552453583151805)


In [183]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :2048
(0.5013750204626395, 7.376683800384242e-05)
(0.31763787628436574, 8.58140743072429e-05)


In [184]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :4096
(0.8958883249029821, 4.598886072111278e-05)
(0.4245485718152961, 0.005489447684038056)


In [185]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 8192, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :8192
(1.6778252988445514, 0.00021309224665019196)
(0.43759020676418225, 2.2458380243482643e-05)


In [186]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 170, prefix: 145, batch :1024
(0.5063307771877367, 5.1432123278614074e-05)
(0.32165159133015847, 0.0001594177773673537)


In [187]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 170, prefix: 145, batch :2048
(0.8833106926509312, 4.89245135826085e-05)
(0.3154308549603637, 7.272906586634951e-05)


In [188]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 170, prefix: 145, batch :4096
(1.6419542711608264, 3.202589260592929e-05)
(0.4081521645492437, 5.06888230663401e-05)


In [189]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 8192, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 170, prefix: 145, batch :8192
(3.1967451621075065, 2.2709015509819746e-05)
(0.645053390945707, 1.1632349238021235e-05)


In [8]:
gen_and_test(num_field = 100,prefix = 10, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 10, batch :4096
(0.5595957533437379, 6.193883207898982e-05)
(0.5608617152486529, 5.863843150407594e-05)


In [9]:
gen_and_test(num_field = 100,prefix = 20, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 20, batch :4096
(0.5620457134684738, 5.6651374792935765e-05)
(0.5199281628034553, 5.123814907960637e-05)


In [10]:
gen_and_test(num_field = 100,prefix = 30, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 30, batch :4096
(0.5636643268624131, 6.791266955648203e-05)
(0.4783232659101486, 8.674929349680784e-05)


In [11]:
gen_and_test(num_field = 100,prefix = 40, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 40, batch :4096
(0.5646622077542909, 0.00018410211768763924)
(0.4299503677353567, 5.715071718630624e-05)


In [12]:
gen_and_test(num_field = 100,prefix = 50, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 50, batch :4096
(0.5591951036939815, 7.220937585863527e-05)
(0.38230759027052896, 4.268407517602069e-05)


In [13]:
gen_and_test(num_field = 100,prefix = 60, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 60, batch :4096
(0.5582984491270415, 5.9912076457300206e-05)
(0.3344013058409399, 5.036184380358841e-05)


In [14]:
gen_and_test(num_field = 100,prefix = 70, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 70, batch :4096
(0.5579624492294935, 5.575926001768646e-05)
(0.29191020343984875, 0.00013983865455971463)


In [15]:
gen_and_test(num_field = 100,prefix = 80, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 80, batch :4096
(0.5561453067526525, 5.214598931123871e-05)
(0.2424685707201763, 9.394493032246283e-05)


In [16]:
gen_and_test(num_field = 100,prefix = 90, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 90, batch :4096
(0.5556173081300697, 4.294864832436149e-05)
(0.23668244891628928, 0.00010493106790542141)


In [17]:
gen_and_test(num_field = 100,prefix = 99, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 99, batch :4096
(0.5562044077990006, 8.147495784594371e-05)
(0.22071542834140817, 0.00011832999222558469)
